In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2023-10-02 17:12:58.425131: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 17:12:58.443737: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 17:12:58.443751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 17:12:58.443766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 17:12:58.447747: I tensorflow/core/platform/cpu_feature_g

In [2]:
class PINN(tf.keras.Model):
    def __init__(self, layers, regularizer_rate=1e-4):
        super(PINN, self).__init__()
        self.nn_layers = layers  # Cambié "layers" a "nn_layers" para evitar conflictos
        self.regularizer = tf.keras.regularizers.l2(regularizer_rate)
        self.nn_model = self.build_model()

    def build_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(1,)))
        for width in self.nn_layers[1:-1]:  # Iterar hasta la penúltima capa para agregar BatchNorm antes de la última capa
            model.add(tf.keras.layers.Dense(width, activation=tf.nn.swish,  # Cambiado a función de activación swish
                                            kernel_initializer='he_normal', 
                                            kernel_regularizer=self.regularizer))
            model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(self.nn_layers[-1], activation=tf.nn.tanh,  # Activación tanh para la última capa
                                        kernel_initializer='glorot_normal', 
                                        kernel_regularizer=self.regularizer))
        return model

    def call(self, inputs):
        return self.nn_model(inputs)

In [5]:
def loss_fn(model, t_inputs, outputs, epsilon, u, DL, K, t_p, Cin):
    # Usar el modelo para predecir
    predicted = model(t_inputs)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t_inputs)
        predicted = model(t_inputs)
        q = K * predicted
        dC_dt = tape.gradient(predicted, t_inputs)
        dC_dz = tape.gradient(dC_dt, t_inputs)
        
    # Implementar las ecuaciones diferenciales
    eq1 = dC_dt + (1 - epsilon) / epsilon * K * dC_dt + u / epsilon * dC_dz - DL * dC_dz
    loss_eq1 = tf.reduce_mean(tf.square(eq1))
    
    # Implementar la condición de borde para t = 0
    Cp = tf.where(t_inputs <= t_p, Cin, 0.0)
    loss_boundary = tf.reduce_mean(tf.square(predicted - Cp))
    
    # Agregar término de regularización
    loss_reg = tf.abs(DL) + tf.abs(K)
    
    # Combinar las pérdidas
    combined_loss = loss_eq1 + loss_boundary + 0.01 * loss_reg  # El factor 0.01 es arbitrario y puede ser ajustado
    
    return combined_loss


In [6]:
# Definición de hiperparámetros y otros valores conocidos
layers = [1, 30, 30, 30, 30, 1]  # Aumentar la complejidad del modelo
model = PINN(layers)

# Decaimiento exponencial de la tasa de aprendizaje
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001, 
    decay_steps=1000, 
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

epsilon = 0.37  # Porosidad del lecho
u = 5.55  # Velocidad superficial del fluido
t_p = 1.0  # Tiempo del pulso de inyección
Cin = 523.23  # Concentración del pulso de analito inyectada en la muestra

# Inicializar DL y K como variables entrenables
DL = tf.Variable(initial_value=0.01, trainable=True, dtype=tf.float32)
K = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32)

# Cargar datos de entrenamiento desde el Excel
data = pd.read_excel("datos_cromatografia.xlsx", sheet_name="training, Cin = 523.23 mg L^-1")

# Normalizar los datos de entrenamiento
t_train = tf.convert_to_tensor(data['min'].to_numpy().reshape(-1, 1) / data['min'].max(), dtype=tf.float32)  # tiempo
C_train = tf.convert_to_tensor(data['AU'].to_numpy().reshape(-1, 1) / data['AU'].max(), dtype=tf.float32)   # concentración

# Entrenamiento
epochs = 15000  # Incrementar el número de épocas

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(model, t_train, C_train, epsilon, u, DL, K, t_p, Cin)
    grads = tape.gradient(loss_value, model.trainable_variables + [DL, K])
    optimizer.apply_gradients(zip(grads, model.trainable_variables + [DL, K]))
    
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch}, Loss: {loss_value.numpy()}, DL: {DL.numpy()}, K: {K.numpy()}")


Epoch: 0, Loss: 274182.96875, DL: 0.01100002322345972, K: 0.9989999532699585
Epoch: 1000, Loss: 272724.21875, DL: 0.044344935566186905, K: 1.022100567817688
Epoch: 2000, Loss: 272724.15625, DL: 0.03475075960159302, K: 1.0076555013656616
Epoch: 3000, Loss: 272724.15625, DL: 0.01916314847767353, K: 0.9842893481254578
Epoch: 4000, Loss: 272724.15625, DL: -2.889390771088074e-08, K: 0.9486587047576904
Epoch: 5000, Loss: 272724.15625, DL: 8.547674951842055e-06, K: 0.8953828811645508
Epoch: 6000, Loss: 272724.15625, DL: -5.308341769705294e-06, K: 0.8165578246116638
Epoch: 7000, Loss: 272724.15625, DL: -2.2567484847968444e-05, K: 0.7014736533164978
Epoch: 8000, Loss: 272724.15625, DL: -2.9378374165389687e-05, K: 0.5381103754043579
Epoch: 9000, Loss: 272724.15625, DL: -3.844232196570374e-05, K: 0.3195743262767792
Epoch: 10000, Loss: 272724.15625, DL: -4.7303798055509105e-05, K: 0.05494518578052521
Epoch: 11000, Loss: 272724.15625, DL: -5.2347360906424e-05, K: 3.0501796572934836e-05
Epoch: 12000